News Prediction System just using Headline(title)

In [88]:
import numpy as numpy
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [91]:

news_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FakeNewsNet.csv')

In [92]:
news_df.shape

(23196, 5)

In [93]:
news_df.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [94]:
news_df.head(3)

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1


In [6]:
# Label Encoding
# news_df.replace({'label': {'FAKE': 1, 'REAL': 0}}, inplace=True)

/tmp/ipython-input-6-3118298064.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  news_df.replace({'label': {'FAKE': 1, 'REAL': 0}}, inplace=True)


In [7]:
news_df.head(3)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0


1 => FAKE

0 => REAL

In [95]:
news_df.isnull().sum()


,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [96]:
# Droping unnecessary colum

news_df.drop(columns = ['news_url', 'source_domain', 'tweet_num'], inplace=True)

In [97]:
news_df.isnull().sum()


,0
title,0
real,0


## Merging the title and text column

In [98]:
news_df['content'] = news_df['title']

In [99]:
news_df

,title,real,content
0,Kandi Burruss Explodes Over Rape Accusation on...,1,Kandi Burruss Explodes Over Rape Accusation on...
1,People's Choice Awards 2018: The best red carp...,1,People's Choice Awards 2018: The best red carp...
2,Sophia Bush Sends Sweet Birthday Message to 'O...,1,Sophia Bush Sends Sweet Birthday Message to 'O...
3,Colombian singer Maluma sparks rumours of inap...,1,Colombian singer Maluma sparks rumours of inap...
4,Gossip Girl 10 Years Later: How Upper East Sid...,1,Gossip Girl 10 Years Later: How Upper East Sid...
...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,1,Pippa Middleton wedding: In case you missed it...
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,0,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...
23193,Jessica Chastain Recalls the Moment Her Mother...,1,Jessica Chastain Recalls the Moment Her Mother...
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",0,"Tristan Thompson Feels ""Dumped"" After Khloé Ka..."


In [101]:
news_df.drop(columns=['title' ], inplace=True)

In [102]:
news_df

,real,content
0,1,Kandi Burruss Explodes Over Rape Accusation on...
1,1,People's Choice Awards 2018: The best red carp...
2,1,Sophia Bush Sends Sweet Birthday Message to 'O...
3,1,Colombian singer Maluma sparks rumours of inap...
4,1,Gossip Girl 10 Years Later: How Upper East Sid...
...,...,...
23191,1,Pippa Middleton wedding: In case you missed it...
23192,0,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...
23193,1,Jessica Chastain Recalls the Moment Her Mother...
23194,0,"Tristan Thompson Feels ""Dumped"" After Khloé Ka..."


In [103]:
news_df['content'][0]

"Kandi Burruss Explodes Over Rape Accusation on 'Real Housewives of Atlanta' Reunion (Video)"

In [104]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [105]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## Stemming Process. The process of taking the root word

In [106]:
ps = PorterStemmer()

In [107]:
def stemming(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)   # remove punctuation and special chars
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # remove extra spaces
    words = sentence.split()                          # split into words
    filtered = [ps.stem(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(filtered)

In [108]:
news_df['content'] = news_df['content'].apply(stemming)

In [109]:
news_df

,real,content
0,1,kandi burruss explod rape accus real housew at...
1,1,peopl choic award best red carpet look
2,1,sophia bush send sweet birthday messag one tre...
3,1,colombian singer maluma spark rumour inappropr...
4,1,gossip girl year later upper east sider shock ...
...,...,...
23191,1,pippa middleton wed case miss itpippa marri la...
23192,0,zayn malik gigi hadid shock split there chanc ...
23193,1,jessica chastain recal moment mother boyfriend...
23194,0,tristan thompson feel dump khlo kardashian ref...


## Seperating the data and label

In [110]:
 X = news_df['content'].values
 Y = news_df['real'].values

In [111]:
print(X.shape, Y.shape)

(23196,) (23196,)


In [112]:
print(X,Y)

['kandi burruss explod rape accus real housew atlanta reunion video'
 'peopl choic award best red carpet look'
 'sophia bush send sweet birthday messag one tree hill costar hilari burton breyton eva'
 ... 'jessica chastain recal moment mother boyfriend slap kick genit'
 'tristan thompson feel dump khlo kardashian refus let move la home exclus'
 'kelli clarkson perform medley kendrick lamar humbl hit billboard music award'] [1 1 1 ... 1 0 1]


Convertint the text into Vector we are using TFIDFVectorizer

In [113]:
# Converting into vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [114]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 183394 stored elements and shape (23196, 13584)>
  Coords	Values
  (0, 55)	0.27102016668711026
  (0, 643)	0.3207618983327733
  (0, 1656)	0.4112148591333234
  (0, 3994)	0.37052462149958415
  (0, 5654)	0.2729512344986755
  (0, 6339)	0.4043694032366516
  (0, 9720)	0.31617973794806686
  (0, 9772)	0.24436158796251867
  (0, 10047)	0.26596502551376666
  (0, 12928)	0.22099122261318518
  (1, 715)	0.31612648659286285
  (1, 1078)	0.34852645646939306
  (1, 1857)	0.3984960712245395
  (1, 2136)	0.43271532701045184
  (1, 7065)	0.3430876010063045
  (1, 8979)	0.4057949050161316
  (1, 9834)	0.38752170739569225
  (2, 1167)	0.1962387693700738
  (2, 1482)	0.38009847068864183
  (2, 1661)	0.3341297238748578
  (2, 1662)	0.27017339957141223
  (2, 2582)	0.24001175378116937
  (2, 3896)	0.25350500648725593
  (2, 5492)	0.22749287146925762
  (2, 5497)	0.24383306501968255
  :	:
  (23193, 9799)	0.32411819090456695
  (23193, 11084)	0.39611815924331306
  (23

In [115]:


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 2, stratify = Y )

In [116]:
print(X_train.shape, X_test.shape, X.shape)

(18556, 13584) (4640, 13584) (23196, 13584)


In [117]:
model = LogisticRegression()

In [118]:
model.fit(X_train, Y_train)

LogisticRegression()

## Evaluation

In [119]:
X_tain_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_tain_prediction, Y_train)

In [120]:
print("X_train_accuracy: ", training_data_accuracy)

X_train_accuracy:  0.8604763957749515


In [121]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [122]:
print("X_test_accuracy: ", testing_data_accuracy)

X_test_accuracy:  0.8338362068965517


## Making a Predictive System

In [123]:
X_new = X_test[42]

prediction = model.predict(X_new)

print(prediction)

if(prediction[0] == 0):
  print("Real")
else:
  print("Fake")

[0]
Real


In [124]:
print(Y_test[42])

0


Taking out important things to perfomr in real world

In [133]:
news = "Are Katie Holmes & Jamie Foxx Expecting A Baby? New Report Claims They Want To Start A Family"

news = stemming(news)   # INclude stemming

vector_trans = vectorizer.transform([news])

prediction_test = model.predict(vector_trans)
if(prediction_test == 0):
  print("Fake")
else:
  print("Real")
print(prediction_test)

Fake
[0]


In [126]:
import pickle

with open('model.pkl', 'wb') as f:
  pickle.dump(model, f)

with open('vectorizer.pkl', 'wb') as f:
  pickle.dump(vectorizer, f)